In [36]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./webdriver/chromedriver')
# chromedriver_path = "chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

rating_df = pd.DataFrame()
restaurant_df = pd.DataFrame()

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    search('맛집')

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 전체 페이지
    while True:
        try:
#             driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
#             sleep(1)
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
                crawling(place, place_lists)
                
                # 다음 페이지 넘기기
                if i==5:
                    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

        except ElementNotInteractableException:
            print('end page')
            break
#         finally:
#             search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    
    global restaurant_df

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_local = place.select('.info_item > .addr > .lot_number')[0].text
        place_category = place.select('.head_item > .subcategory')[0].text
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail
        
        row = {"ItemID":place_name, "address": place_address, "local" : place_local, "category": place_category}
        row = pd.DataFrame(row, index=[1])
        restaurant_df = restaurant_df.append(row, ignore_index=True)
        
        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)

#         detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
#         driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
#         driver.switch_to.window(driver.window_handles[-1])  
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
                
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver, rating_df

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
            timestamp = review.select(' div > span.time_write') #시간정보
            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)
                
                
                try:
                    if(user_id == NULL and rating ==NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    elif(user_id == NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    elif(rating ==NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    else:
                        row = {"ItemID":place_name, "UserID":user_id[0].get('data-userid'), "review" : comment[0].text,
                               "Rating":rating[0].text.replace('점', ''), "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)    
            
                except:
                    row = {"ItemID":place_name, "UserID":None, "review" : None,
                           "Rating": None, "Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)                
                
                
#                 try:
#                     row = {"ItemID":place_name, "UserID":user_id[0].get('data-userid'), "review" : comment[0].text,
#                            "Rating":rating[0].text.replace('점', ''), "Timestamp":timestamp[0].text}
#                     row = pd.DataFrame(row, index=[i])
#                     rating_df = rating_df.append(row, ignore_index=True)
            
#                 except:
#                     row = {"ItemID":place_name, "UserID":None, "review" : None,
#                            "Rating": None, "Timestamp":timestamp[0].text}
#                     row = pd.DataFrame(row, index=[i])
#                     rating_df = rating_df.append(row,ignore_index=True)
                
                
    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

#### 조연탄
헉 가게가 좀 부산스럽고 종업원분들이 극 인싸이싱거같아요 그거랑 별개로 맛은 정말 좋습니다/5
어엄청 친절하셔유! 맛있어여 또 갈거예요/5
/5
찐맛집/5
내 최애 고기집/5
말모... 존맛/5
/5
육즙에 향긋한 고기.  소주와 맥주를 다른 온도에서 얼려 살얼음 만들어 망치로 팡팡 두들겨 주시는 술, 맬젓에 비빈 밥, 냉면, 만두, 묵은지 베이스 국물의 김치, 고기 소스 조합까지 완벽그자체 다 최고예요 /5
고기가 저의 스타일이 아니였어여 나머지 메뉴는 맛있었어여 비냉.솥밥 다괜찮았어여 술을부르는안주가많았어여/2
저는 그냥 쏘쏘하네요 ^^~/2
맞은편 2호점에 사장님인지 점원인지 손님와도 쳐다도 안보고 해서 둘이라고 했는대 무시하고 자기일해서 다른직원한테 저기앉으면되냐니까 본점가라고 하는데 아 웨이팅해야 되요?하니까 가보면 안다면서 엄청바쁜것도 아닌데 기본적인 친절은 없네요 핸드폰 충전되냐니까 일단 나가고 앉으면 다시말하래요 ㅋㅋㅋ 본점이란데는 웨이팅하는사람 추위에 떨고있는데 난로 꺼져있고 방문자.../1
원래 오랜기간 목동 원조소금구이집 다니다가 하도 사람이 많길래 궁금해서 와보니 와보길 잘했다싶게 맛좋고 곁드림 소스와 찬들이 제대로 써포트해주네./4
1점도 아까움/1
너무 맛있어요 비계가 좀 많긴 하지만 그래도/5
가격에 비해 양은 적지만 고기가 맛있고 사이드가 뭐 하나 빠지지 않고 맛있어요! 고기도 잘라줘서 편하게 (직접 먹여줄 때도 있어 낯 가리면 부담될 수도) 먹을 수 있고 일하시는 분들 분위기가 유쾌합니다. 고기보다 사이드가 더더 맛있음!!/4
고기 맛있어요. 요즘엔 자주 못가는데 또 가고 싶음/5
웨이팅 할 정도로 맛있어요 소스와 반찬들과 전반적인 조화도 너무 좋구요! 돼지 껍질에 털이 보이는 미스정도? 소주를 얼음보틀에 담아주는 세심함까지 최고였어요! 그런데 맛이 좋음에도 바쁜시간대에 갔더니 직원분들이 엄청 뿔이나 있더라구요 웨이팅까지 해가며서 맛있게 먹으러 갔는데 직원분들 기분 좋은지 아닌지 주문하는거 무시(?)당해가면서 까지

In [44]:
rating_df.head()

,ItemID,UserID,review,Rating,Timestamp
0,조연탄,k9hsnf,헉 가게가 좀 부산스럽고 종업원분들이 극 인싸이싱거같아요 그거랑 별개로 맛은 정말 ...,5,2021.08.03.
1,조연탄,o9qv2m,어엄청 친절하셔유! 맛있어여 또 갈거예요,5,2021.06.25.
2,조연탄,1cbc382,,5,2021.06.23.
3,조연탄,13n8k41,찐맛집,5,2021.06.19.
4,조연탄,1fio525,내 최애 고기집,5,2021.06.15.


In [43]:
restaurant_df.head(20)

,ItemID,address,local,category
0,조연탄,서울 강서구 곰달래로60길 29,(지번) 화곡동 782-12,"육류,고기"
1,먹자대게,서울 양천구 목동서로 213 세신비젼프라자 1층 108호,(지번) 목동 923,"게,대게"
2,착한낙지 목동점,서울 양천구 국회대로 289,(지번) 목동 802-9,"해물,생선"
3,일미락 목동본점,서울 양천구 목동동로 226-16 1층,(지번) 목동 406-126,"육류,고기"
4,원조소금구이,서울 강서구 곰달래로60길 5 1층,(지번) 화곡동 781-12,"육류,고기"
5,엉털네숯불꼼장어,서울 양천구 등촌로 36,(지번) 목동 792-2,장어
6,더아리엘 목동점,서울 양천구 오목로 300 현대하이페리온2차 206동 지하1층 109~115호,(지번) 목동 961-1,뷔페
7,양천옥설렁탕,서울 양천구 등촌로 86,(지번) 목동 730-2,설렁탕
8,오목집 목동본점,서울 양천구 목동서로 155 목동파라곤 지하상가 45호,(지번) 목동 917,"족발,보쌈"
9,스타벅스 화곡DT점,서울 강서구 등촌로 57,(지번) 화곡동 772-67,커피전문점


In [40]:
rating_df.to_csv('rating_df.csv')

In [51]:
rating_df.to_csv('rating_df_test.csv', sep=',', na_rep='NaN', encoding='utf-8-sig')

In [48]:
rating_df.to_csv('rating_df_ko.csv', encoding='utf-8-sig')

In [39]:
restaurant_df.to_csv('restaurant_df.csv')

In [53]:
restaurant_df.to_csv('restaurant_df_test.csv', sep=',', na_rep='NaN', encoding='utf-8-sig')

In [49]:
restaurant_df.to_csv('restaurant_df_ko.csv' , encoding='utf-8-sig')

In [ ]:
import os
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./webdriver/chromedriver')
# chromedriver_path = "chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

rating_df = pd.DataFrame() # 리뷰 저장 데이터 프레임
restaurant_df = pd.DataFrame() #음식점 정보 저장

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    search('맛집')

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail

        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)

#         detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
#         driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
#         driver.switch_to.window(driver.window_handles[-1])  
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점

            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()